<a href="https://colab.research.google.com/github/mitramir55/PassivePy/blob/main/PassivePy%20notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# first run this cell to install the package


!pip install -r https://raw.githubusercontent.com/mitramir55/PassivePy/main/PassivePyCode/PassivePySrc/requirements_lg.txt
!pip install PassivePy==0.2.21

  Using cached en_core_web_sm-3.4.0-py3-none-any.whl



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# then run this to import the package and create PassivePy
import pandas as pd
import os
import regex as re
pd.set_option('display.max_colwidth', None)
import string
from PassivePySrc import PassivePy



passivepy = PassivePy.PassivePyAnalyzer(spacy_model = "en_core_web_lg")

## Single sentence example

In [3]:
# Try changing the sentence below:
sample_text = "Bears are dreamt of in your fantasies!"
resutl_1 = passivepy.match_text(sample_text, full_passive=True, truncated_passive=True)
resutl_1

,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary,full_passive_matches,full_passive_count,full_passive_sents_count,full_passive_percentages,binary_full_passive,truncated_passive_matches,truncated_passive_count,truncated_passive_sents_count,truncated_passive_percentages,binary_truncated_passive
0,Bears are dreamt of in your fantasies!,1,[are dreamt],1,1,1.0,1,[],0,0,0.0,0,[are dreamt],1,1,1.0,1


In [4]:
# parse each senence 
# POS: coarse-grained part of speech tag
# dependency: dependency tag
# tag: fine-grained part of speech tags
# lemma: canonical form of the word


table_1 = passivepy.parse_sentence(sample_text)
table_1

,Bears,are,dreamt,of,in,your,fantasies,!
POS,NOUN,AUX,VERB,ADP,ADP,PRON,NOUN,PUNCT
dependency,nsubj,auxpass,ROOT,prep,prep,poss,pobj,punct
tag,NNS,VBP,VBN,IN,IN,PRP$,NNS,.
lemma,bear,be,dream,of,in,your,fantasy,!


In [6]:
table_2 = passivepy.parse_sentence("Book was read by him.")
table_2

,Book,was,read,by,him,.
POS,PROPN,AUX,VERB,ADP,PRON,PUNCT
dependency,nsubjpass,auxpass,ROOT,agent,pobj,punct
tag,NNP,VBD,VBN,IN,PRP,.
lemma,Book,be,read,by,he,.


In [10]:
table_2.to_csv('table_2.csv', index=True)

# import a dataset from your colab
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# or use the sample dataset in our github

!git clone https://github.com/mitramir55/PassivePy.git

Cloning into 'PassivePy'...


In [6]:
df = pd.read_csv('PassivePy\Data\crowd_source_dataset.csv')
df.head(1)

,Sentence,ID,Test,Cond,human_coding
0,The politics being discussed were causing scene.,A3ATU3VULAB7K2,2.0,2,1


In [ ]:
df = pd.read_csv('PassivePy\Data\crowd_source_dataset.csv')
df.head(1)

In [28]:
df_without = pd.read_csv(r'D:\Papers\Paper 3 - Recommender Systems\additional on passivepy\cola_analyzed_without.csv')
df_without.head(1)

,Unnamed: 0,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary,uid
0,0,The sailors rode the breeze clear of the rocks.,1,[],0,0,0.0,0,1


In [30]:
df_with = df_without[['document', 'uid']]

## Sentence-level from a dataset

In [15]:
# with sentence-level analysis, we see each record as a document 
# that needs to be separated into sentences and then be analyzed.


df_detected_s = passivepy.match_sentence_level(df, column_name='Sentence', n_process = 1,
                                                batch_size = 1000, add_other_columns=True,
                                                truncated_passive=False, full_passive=False)
df_detected_s

Detecting Sentences...


  0%|          | 0/1150 [00:00<?, ?it/s]

Total number of sentences = 1156
Starting to find passives...


  0%|          | 0/1156 [00:00<?, ?it/s]

,docId,sentenceId,sentences,all_passives,all_passives_count,binary,ID,Test,Cond,human_coding
0,1,1,the politics being discussed were causing scene.,[being discussed],1,1,A3ATU3VULAB7K2,2.0,2,1
1,2,1,the world of politics is quite fascinating.,None,0,0,A1CTJ8UT7SF317,1.0,1,0
2,3,1,i don usually participate in discussions about politics.,None,0,0,A327Q0Y175MEH7,1.0,1,0
3,4,1,the president was impeached before his second term.,[was impeached],1,1,A19TD2J8506A4Y,1.0,2,1
4,5,1,the president just gave speech about covid relief.,None,0,0,A2DDXYTEVDV5F5,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...
1151,1146,1,the shareholders were not satisfied by the ceo explanations,None,0,0,A1EDKVYT9ZCGUH,2.0,2,1
1152,1147,1,i would mind what you are doing in this meeting.,None,0,0,A3CX8S6Q0QEW8E,1.0,1,0
1153,1148,1,i had no business doing that.,None,0,0,AYX9OCFWBM939,2.0,2,0
1154,1149,1,they had good business sales then.,None,0,0,A1HNB2243A2PQH,2.0,2,0


## Corpus level


In [37]:
df_detected_c = passivepy.match_corpus_level(df, column_name='Sentence', n_process = 1,
                                            batch_size = 1000, add_other_columns=True,
                                            truncated_passive=False, full_passive=False)
df_detected_c

Detecting Sentences...
Starting to find passives...


,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary,uid
0,The sailors rode the breeze clear of the rocks.,1,[],0,0,0.0,0,1
1,The weights made the rope stretch over the pulley.,1,[],0,0,0.0,0,2
2,The mechanical doll wriggled itself loose.,1,[],0,0,0.0,0,3
3,"If you had eaten more, you would want less.",1,[],0,0,0.0,0,4
4,"As you eat the most, you want the least.",1,[],0,0,0.0,0,5
...,...,...,...,...,...,...,...,...
1038,John considers Bill silly.,1,[],0,0,0.0,0,1039
1039,John considers Bill to be silly.,1,[],0,0,0.0,0,1040
1040,John bought a dog for himself to play with.,1,[],0,0,0.0,0,1041
1041,John arranged for himself to get the prize.,1,[],0,0,0.0,0,1042


In [21]:
df_without = pd.read_csv(r"D:\Papers\Paper 3 - Recommender Systems\additional on passivepy\re_analysis_crowdsource.csv")
df_without.head()

,Unnamed: 0,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary
0,0,The politics being discussed were causing scene.,1,['being discussed'],1,1,1.0,1
1,1,The world of politics is quite fascinating.,1,[],0,0,0.0,0
2,2,I don usually participate in discussions about politics.,1,[],0,0,0.0,0
3,3,The president was impeached before his second term.,1,['was impeached'],1,1,1.0,1
4,4,The President just gave speech about COVID relief.,1,[],0,0,0.0,0


In [40]:
df_merged = pd.merge(df_without, df_detected_c, on='uid', suffixes=('_without', '_with'))
df_merged

,Unnamed: 0,document_without,count_sents_without,all_passives_without,passive_count_without,passive_sents_count_without,passive_percentages_without,binary_without,uid,document_with,count_sents_with,all_passives_with,passive_count_with,passive_sents_count_with,passive_percentages_with,binary_with
0,0,The sailors rode the breeze clear of the rocks.,1,[],0,0,0.0,0,1,The sailors rode the breeze clear of the rocks.,1,[],0,0,0.0,0
1,1,The weights made the rope stretch over the pulley.,1,[],0,0,0.0,0,2,The weights made the rope stretch over the pulley.,1,[],0,0,0.0,0
2,2,The mechanical doll wriggled itself loose.,1,[],0,0,0.0,0,3,The mechanical doll wriggled itself loose.,1,[],0,0,0.0,0
3,3,"If you had eaten more, you would want less.",1,[],0,0,0.0,0,4,"If you had eaten more, you would want less.",1,[],0,0,0.0,0
4,4,"As you eat the most, you want the least.",1,[],0,0,0.0,0,5,"As you eat the most, you want the least.",1,[],0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,1038,John considers Bill silly.,1,[],0,0,0.0,0,1039,John considers Bill silly.,1,[],0,0,0.0,0
1039,1039,John considers Bill to be silly.,1,[],0,0,0.0,0,1040,John considers Bill to be silly.,1,[],0,0,0.0,0
1040,1040,John bought a dog for himself to play with.,1,[],0,0,0.0,0,1041,John bought a dog for himself to play with.,1,[],0,0,0.0,0
1041,1041,John arranged for himself to get the prize.,1,[],0,0,0.0,0,1042,John arranged for himself to get the prize.,1,[],0,0,0.0,0


In [41]:
df_merged[df_merged['passive_count_without'] != df_merged['passive_count_with']]

,Unnamed: 0,document_without,count_sents_without,all_passives_without,passive_count_without,passive_sents_count_without,passive_percentages_without,binary_without,uid,document_with,count_sents_with,all_passives_with,passive_count_with,passive_sents_count_with,passive_percentages_with,binary_with


In [18]:
len(df_detected_c[df_detected_c['human_coding']==df_detected_c['binary']])/len(df_detected_c) # accuracy in this case

0.9791304347826087

In [1]:
import spacy
from spacy import displacy
spacy_model = "en_core_web_lg"
nlp = spacy.load(spacy_model)

In [2]:
text = 'beautiful day'
doc = nlp(text)
displacy.serve(doc, style = "dep", options={"compact":False})

C:\Users\Lenovo\anaconda3\lib\site-packages\spacy\displacy\__init__.py:97: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
